In [166]:
import pandas as pd
import numpy as np

df = pd.read_csv('questions.csv')

for col in df.columns:
    df[col] = df[col].replace(',', '', regex=True)
    df[col] = df[col].replace('$', '', regex=True)

df['Sr. No'] = df['Sr. No'].fillna(method='ffill')
df['Sr. No'] = df['Sr. No'].astype(int)
grouped_df = df.groupby('Sr. No')
required_columns = ['Data'] + [f'Unnamed: {i}' for i in range(3, 10)]
processed_groups = {}
for name, group in grouped_df:
    missing_columns = [
        col for col in required_columns if col not in group.columns]
    if missing_columns:
        print(f"Missing columns in group {name}: {missing_columns}")
        continue
    filtered_group = group[required_columns]
    filtered_group.fillna('', inplace=True)
    processed_groups[name] = filtered_group
df.drop(columns=required_columns, inplace=True)
new_df = df.groupby('Sr. No').first()

for i in range(1, len(processed_groups) + 1):
    d = processed_groups[i]
    new_columns = d.iloc[0]
    d.columns = new_columns
    d = d.drop(d.index[0])
    d.reset_index(drop=True, inplace=True)
    d.replace("", float("NaN"), inplace=True)
    d.dropna(how='all', axis=1, inplace=True)
    d = d.drop(d.index[-1])
    processed_groups[i] = d

new_df = new_df.assign(Data=processed_groups.values())
df = new_df
df.reset_index(drop=True, inplace=True)
df.head()

C:\Users\Aditya\AppData\Local\Temp\ipykernel_3116\1602288140.py:10: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['Sr. No'] = df['Sr. No'].fillna(method='ffill')
C:\Users\Aditya\AppData\Local\Temp\ipykernel_3116\1602288140.py:22: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  filtered_group.fillna('', inplace=True)
C:\Users\Aditya\AppData\Local\Temp\ipykernel_3116\1602288140.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_group.fillna('', inplace=True)
C:\Users\Aditya\AppData\Local\Temp\ipykernel_3116\1602288140.py:22: FutureWarning: Set

,Questions,Alpha,Type,Data
0,A researcher is studying the effect of differe...,0.05,CRD,0 Fertilizer A Fertilizer B Fertilizer C 0 ...
1,A company wants to compare customer satisfacti...,0.01,CRD,7 Model X Model Y Model Z 0 7 8 ...
2,To study the effect of three different diets o...,0.05,CRD,14 Diet A Diet B Diet C 0 5 4 ...
3,The preference scores (out of 10) for three di...,0.05,FD,21 Market Packaging A Packaging B Packaging...
4,The productivity (in units) for employees usin...,0.05,FD,29 Shift Tool A Tool B Tool C 0 Day...


In [167]:
temp = df.iloc[26]['Data']
temp

198,,,Treatment 2,,,
0,NaN,NaN,A,B,C,D
1,Treatment 1,A,1.2,2.2,1.7,2.4
2,NaN,B,1.9,2.7,1.9,2.8


In [168]:
def isFirstColumnMergedHeader(df):
    firstColumn = df.to_numpy()
    firstColumn = firstColumn.T
    firstColumn = list(set(firstColumn[0]))
    firstColumn = [x for x in firstColumn if not pd.isna(x)]
    return len(firstColumn) == 1

isFirstColumnMergedHeader(temp)

True

In [169]:
def isSecondColumnGroupedHeader(df):
    column = df.to_numpy()
    column = column.T
    column = [x for x in column[1] if not pd.isna(x)]
    sample = list(set(column))
    countOfSample = [column.count(x) for x in sample]
    if len(set(countOfSample)) == 1 and countOfSample[0] > 1:
        return True
    else:
        return False

isSecondColumnGroupedHeader(temp)

ValueError: min() arg is an empty sequence

In [ ]:
def isFirstColumnHeader(df):
    if isFirstColumnMergedHeader(df):
        return True
    firstColumn = df.to_numpy()
    firstColumn = firstColumn.T[0]
    if any(isinstance(n, str) and n.isalpha() for n in firstColumn):
        return True
    elif all(isinstance(n, str) and n.isnumeric() for n in firstColumn):
        firstColumn = [int(x) for x in firstColumn]
        if firstColumn == list(range(firstColumn[0], firstColumn[-1] + 1)):
            return True
        else:
            return False
    else:
        if all(isinstance(float(x), float) for x in firstColumn):
            return False
        return True
isFirstColumnHeader(temp)

True

In [ ]:
def isFirstColumnGroupedHeader(df):
    if isFirstColumnHeader(df):
        column = df.to_numpy()
        column = column.T[0]
        column = [x for x in column if not pd.isna(x)]
        sample, countOfSample = np.unique(column, return_counts=True)
        if all(x == countOfSample[0] for x in countOfSample) and countOfSample[
                0] > 1:
            return True
        else:
            return False
    else:
        return False


isFirstColumnGroupedHeader(temp)

False

In [ ]:
def isFirstRowHeader(df, data):
    firstRow = df.iloc[0]
    firstRow = list(firstRow)
    firstRow = [x for x in firstRow if not pd.isna(x)]
    if isFirstColumnHeader(data):
        firstRow = firstRow[1:]
    if any(n.isalpha() for n in firstRow):
        return True
    elif all(n.isnumeric() for n in firstRow):
        num_list = [int(n) for n in firstRow]
        if num_list == list(range(num_list[0], num_list[0] + len(num_list))):
            return True
        else:
            return False
    else:
        if all(isinstance(x, (int, float)) for x in firstRow):
            if all(x.is_integer() for x in firstRow):
                return False
            else:
                return True
        else:
            return False

In [ ]:
def extractData(df):
    data = []
    if isFirstColumnMergedHeader(df):
        if isSecondColumnGroupedHeader(df):
            data = df.iloc[:, 2:]
        else:
            data = df.iloc[:, 1:]
    elif isFirstColumnHeader(df):
        data = df.iloc[:, 1:]
    elif isFirstColumnGroupedHeader(df):
        data = df.iloc[:, 1:]
    else:
        data = df.iloc[:, :]

    if isFirstRowHeader(data, df):
        data = data[1:]
    data.reset_index(drop=True, inplace=True)
    data = data.to_numpy()
    for i in range(len(data)):
        for j in range(len(data[i])):
            try:
                data[i][j] = float(data[i][j])
            except:
                pass
    return data

extractData(temp)


array([[1.2, 2.2, 1.7, 2.4],
       [1.9, 2.7, 1.9, 2.8]], dtype=object)